## Advent of code 2023 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 12 part 0

def check_nums(s, nums, check_all):
    #print(f'check_nums: {s=}, {nums=}, {check_all=}')
    n=0
    in_dmg=False
    nums_idx=0
    for c in s:
        if in_dmg and c=='#':
            n+=1
        elif (not in_dmg) and c=='.':
            n=0
        elif c=='#':
            n=1
            in_dmg=True
        else:
            assert c=='.'
            if n>0:
                if nums_idx>=len(nums):
                    return False
                if nums[nums_idx]==n:
                    nums_idx+=1
                else:
                    return False
            n=0
    if check_all:
        if n>0:
            if nums_idx>=len(nums):
                return False
            if nums[nums_idx]==n:
                nums_idx+=1
            else:
                return False
        return nums_idx>=len(nums)
    else:
        return True

In [ ]:
assert check_nums('.###....#', [3, 2, 1], True)==False
assert check_nums('.###....#', [3, 2, 1], False)==True
assert check_nums('.###....##.#', [3, 2, 1], True)==True
assert check_nums('.###....##.#.#.#.#', [3, 2, 1], False)==False

In [ ]:
# 2023 day 12 part 1
# mv ~/Downloads/input* data_src/2023-day-12-input.txt
# big input file looks like: 1000 short lines
# idea: part 1 parse ..., then ...

sample2='''
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
'''

def check_arran(s, line, nums):
    if len(s)>=len(line):
        if not check_nums(s, nums, True):
            return 0
        return 1
    else:
        if not check_nums(s, nums, False):
            return 0
        c=line[len(s)]
        if c=='?':
            total=0
            total+=check_arran(s+'.', line, nums)
            total+=check_arran(s+'#', line, nums)
            return total
        else:
            assert c=='.' or c=='#'
            total=check_arran(s+c, line, nums)
            return total

def count_arran(line, nums):
    '''count possible arrangements'''
    cnt=check_arran('', line, nums)
    #print(f'count_arran: {line=}, {cnt=}')
    return cnt

sample1=open('data_src/2023-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[]
for line in lines:
    tup=line.split()
    assert len(tup)==2
    nums=[ int(n) for n in tup[1].split(',') ]
    data.append( (tup[0], nums) )

In [ ]:
# 2023 day 12 part 1 run

total=0
for tup in data:
    total+=count_arran(*tup)
print(f'part 1: {total=}')

In [ ]:
# 2023 day 12 part 2
# idea: DFS as implemented above takes too long, 
# top 3 lines of actual input:
#  .#?#???????.????# 1,2,3,2,1  # cnt=49762
#  ?????????? 1,1,4
#  ????.??.??.??? 1,2
# unfolded already take way too long this way
# - could work from nums generating sequences of 0-n (later 1-n) dots, followed by n #, see if they match
#   the string, faster but still DFS
# - you don't have to generate them to count them? should take second exmaple above, not unfolded,
#   write out solutions by generating sequences, and consider the ?###???????? 3,2,1 example
#   from part 1 as well

def unfold_tup(line, nums):
    s=line
    nums2=list(nums)
    for _ in range(4):
        s+='?'+line
        nums2.extend(nums)
    return s, nums2

#unfold_tup('.#', [1])
#unfold_tup('???.###', [1,1,3])

total=0
for tup in data:
    line,nums=unfold_tup(*tup)
    cnt=count_arran(line, nums)
    print(f'count_arran2: {line=}, {cnt=}')
    total+=cnt
print(f'part 2: {total=}')

In [ ]:
# 2023 day 11 part 1
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: big map
# idea: part 1 parse as list of lines, then expand in the lines,
# then convert to a list of coordinates and calculate manhattan distance

sample2='''
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# first expand columns
x=0
while x<len(lines[0]):
    foundgal=False
    for y in range(len(lines)):
        if lines[y][x]=='#':
            foundgal=True
    if not foundgal:
        for y in range(len(lines)):
            lines[y]=lines[y][:x]+' '+lines[y][x:]
        x+=2
    else:
        x+=1
# expand rows
y=0
while y<len(lines):
    foundgal='#' in lines[y]
    if not foundgal:
        lines.insert(y, '')
        y+=2
    else:
        y+=1

#print('expanded:')
#for line in lines:
#    print(line)

def manh_dist(a, b):
    '''manhattan distance between two positions, each a tuple of x,y'''
    dist=abs(a[0]-b[0])+abs(a[1]-b[1])
    return dist

def total_dist(data):
    '''based on list of x,y positions calculate total manhattan distance between
    each pair'''
    total=0
    for pair in itertools.combinations(data, 2):
        total+=manh_dist(*pair)
    return total

data=[] # (x,y) of galaxy
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            data.append( (x,y) )
total=total_dist(data)
print(f'part 1: {total=}')

# part 1: 10033566

In [ ]:
# 2023 day 11 part 2
# idea: based on lists of empty row and column numbers, shift/expand each galaxy
# while converting from the map to list of coordinates, then calculate total distance 
# as before

def empty_rows(lines):
    '''return list of empty row numbers'''
    return [ y for y,row in enumerate(lines) if '#' not in row ]

def empty_cols(lines):
    '''return list of empty column numbers'''
    return [ x for x in range(len(lines[0])) if '#' not in { lines[y][x] for y in range(len(lines)) } ]

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
erows=empty_rows(lines)
ecols=empty_cols(lines)
print(f'erows: {erows}')
print(f'ecols: {ecols}')

data=[] # (x,y) of galaxy, expanded
fact=1000000
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            newx=x
            for n in ecols:
                if n<x:
                    newx+=fact-1
            newy=y
            for n in erows:
                if n<y:
                    newy+=fact-1
            data.append( (newx,newy) )
total=total_dist(data)
print(f'part 2: {total=}')

In [ ]:
# TEMPLATE
# 2023 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed